In [1]:
import os
import sys
import dask
import uproot4
import numpy as np
import pandas as pd
from hist import Hist
import hist
import uproot4 as uproot
from dask.array import histogram as ds_hist
import dask.dataframe as dd
import timeit
from dask.distributed import Client
import storage_functions as sf
import time
import psutil
##procedurre to make client dask work
# read hdf data convert it to csv and save it, then read csv with dask data frame
# or just reset index and then go from there
# 2tb of data run 2gb per job each job takes half a day
# try load pickle create new files
import re
import subprocess
from pympler import asizeof

In [2]:
rootdir = "/Skl_Data/"


In [3]:
os.walk(rootdir)

<generator object walk at 0x7f17e04d3728>

In [4]:
full_path = str(subprocess.check_output(['pwd']))
path = str(subprocess.check_output(['pwd']))[2:len(full_path)-3]

In [5]:
full_path

"b'/afs/cern.ch/user/k/kmalirz/pythium\\n'"

In [6]:
os.listdir(path+rootdir)

['.ipynb_checkpoints', 'test12345.h5']

In [7]:
regex = re.compile('(?=^[^.].)(.*h5$)|(?=^[^.].)(.*csv$)')
directory_check = re.compile('(?=^[^.].)')
#print(rootdir)
file_names = []
for root, dirs, files in os.walk(path+rootdir,topdown = True):
    dirs[:] = [d for d in dirs if directory_check.match(d)]
    for file in files:
        if regex.match(file):
            file_names.append(root+file)

In [8]:
file_names

['/afs/cern.ch/user/k/kmalirz/pythium/Skl_Data/test12345.h5']

In [9]:
h_names = []
file_names = []
temp = pd.read_hdf("Skl_Data/test12345.h5")

In [10]:
temp

leptons_pt      jets_pt   weight_mc  runNumber       BDT
event lep jet bdt                                                             
0     0   0   0    127083.039062  104784.4375  736.971985     284500  0.344902
              1    127083.039062  104784.4375  736.971985     284500  0.367624
              2    127083.039062  104784.4375  736.971985     284500  0.069904
              3    127083.039062  104784.4375  736.971985     284500  0.010252
              4    127083.039062  104784.4375  736.971985     284500  0.207318
...                          ...          ...         ...        ...       ...
3681  0   3   0     89429.367188   48895.3125  735.537537     284500  0.151289
              1     89429.367188   48895.3125  735.537537     284500  0.329823
              2     89429.367188   48895.3125  735.537537     284500  0.144091
              3     89429.367188   48895.3125  735.537537     284500  0.116522
              4     89429.367188   48895.3125  735.537537     284500  0.258275

[91250 rows x 5 columns]

In [11]:
%%time
x = []
for i in range(0,50):
    x.append(temp)

CPU times: user 13 µs, sys: 6 µs, total: 19 µs
Wall time: 24.6 µs


In [12]:
%%time
temp = pd.concat(x,ignore_index=False,sort = False)

CPU times: user 407 ms, sys: 499 ms, total: 906 ms
Wall time: 894 ms


In [13]:
%%time
for i in range(0,8):
    h_names.append(f"h{i}")
    file_names.append(f"test{i}.h5")
    #if os.path.isfile(f"/eos/user/k/kmalirz/test{i}.pkl"): os.remove(f"/eos/user/k/kmalirz/test{i}.pkl")
    #if os.path.isfile(f"/eos/user/k/kmalirz/test{i}.h5"): os.remove(f"/eos/user/k/kmalirz/test{i}.h5")
    #temp.to_hdf(f"/eos/user/k/kmalirz/test{i}.h5", key='branches')
    #temp.to_pickle(f"/eos/user/k/kmalirz/test{i}.pkl")

CPU times: user 10 µs, sys: 5 µs, total: 15 µs
Wall time: 18.1 µs


In [14]:
file_names

['test0.h5',
 'test1.h5',
 'test2.h5',
 'test3.h5',
 'test4.h5',
 'test5.h5',
 'test6.h5',
 'test7.h5']

In [15]:
histogramming = sf.HistoMaker()

In [16]:
client = histogramming.client_start(n_workers = 8,memory_limit='2GB',threads_per_worker=1)

In [17]:
client

Client Scheduler: tcp://127.0.0.1:45561 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.00 GB


{'h0': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h1': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h2': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h3': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h4': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h5': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h6': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()),
 'h7': Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double())}

In [35]:

x = histogramming.create_file_list(top_directory = '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/')

In [36]:
x[1:20]

['/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305435.Sherpa.DAOD_JETM6.e5037_s3126_r10724_p4308.mc16e_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305435.Sherpa.DAOD_JETM6.e5037_s3126_r9364_p4308.mc16a_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305436.Sherpa.DAOD_JETM6.e5037_s3126_r10724_p4308.mc16e_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305436.Sherpa.DAOD_JETM6.e5037_s3126_r9364_p4308.mc16a_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305437.Sherpa.DAOD_JETM6.e5037_s3126_r9364_p4308.mc16a_gammajet_210921_sys_v2_output_root_0.pkl',
 '/

In [41]:
h_params = {'bins': 300, 'start' : 0, 'stop': 2*10**6,'name': 'test'}
hists = histogramming.create_histograms(drop_existing = True, histogram_names = x[:20], histogram_params = h_params)

In [44]:
list(hists.keys())

['/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305435.Sherpa.DAOD_JETM6.e5037_s3126_r10201_p4308.mc16d_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305435.Sherpa.DAOD_JETM6.e5037_s3126_r10724_p4308.mc16e_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305435.Sherpa.DAOD_JETM6.e5037_s3126_r9364_p4308.mc16a_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305436.Sherpa.DAOD_JETM6.e5037_s3126_r10724_p4308.mc16e_gammajet_210921_sys_v2_output_root_0.pkl',
 '/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305436.Sherpa.DAOD_JETM6.e5037_s3126_r9364_p4308.mc16a_gammajet_210921_sys_v2_output_root_0.pkl',
 '

In [45]:
hists['/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.305442.Sherpa.DAOD_JETM6.e5020_s3126_r10724_p4308.mc16e_gammajet_210921_sys_v2_output_root_3.pkl']

Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double())

In [20]:
time.sleep(2)

In [21]:
os.getcwd()

'/afs/cern.ch/user/k/kmalirz/pythium'

In [39]:
len(x)

2890

In [40]:
x[100]

'/eos/atlas/atlascerngroupdisk/perf-jets/JSS/WTopBackgroundSF2019/UFO_test/slimmed_SEP_v1/user.brle.364678.Sherpa.DAOD_JETM6.e6929_s3126_r10201_p4308.mc16d_dijet_210908_sys_v1_output_root_1.pkl'

In [24]:
temp = pd.read_pickle(x[2000])

In [25]:
temp

,,rljet_L4,rljet_L5,rljet_L2,rljet_topTag_DNN20_TausRatio_inclusive_score,rljet_topTag_DNN20_TausRatio_qqb_score,rljet_pt_comb,rljet_M2,rljet_L3,rljet_L1,rljet_topTag_DNN20_qqb_score,rljet_topTag_DNN19_qqb_score,rljet_topTag_DNN19_inclusive_score,rljet_D2,rljet_N3,rljet_N2,rljet_topTag_DNN20_inclusive_score,rljet_ungroomed_ntrk500,"rljet_m_comb[:,0]"
entry,subentry,,,,,,,,,,,,,,,,,,
0,0,1.478220,0.173853,0.142326,0.233609,0.028133,234297.921875,0.058569,0.049985,0.052006,0.008996,0.144268,0.305211,1.608747,2.194288,0.261934,0.318813,25,77868.570312
1,0,1.159872,0.182679,0.198222,0.065824,0.028205,533714.125000,0.098365,0.097097,0.096210,0.039847,0.186804,0.116970,1.039699,1.467166,0.290440,0.073059,53,256805.593750
2,0,1.424379,0.212927,0.246536,0.084335,0.009025,234563.500000,0.091785,0.067575,0.079675,0.009816,0.098301,0.111656,2.505868,1.881467,0.373857,0.062968,19,77142.257812
3,0,1.553125,0.290454,0.179543,0.515852,0.180849,237934.281250,0.078671,0.072736,0.074003,0.074075,0.298814,0.515191,1.265350,1.748846,0.272025,0.525609,19,97159.187500
4,0,1.127484,0.339977,0.298020,0.384343,0.175337,257513.250000,0.152028,0.136176,0.135310,0.660136,0.783664,0.736589,1.391467,1.362434,0.377992,0.562129,40,169429.921875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5554,0,1.242116,0.224202,0.217022,0.134496,0.031480,272262.781250,0.098071,0.081542,0.085398,0.033329,0.242306,0.195403,1.739679,1.758764,0.342052,0.169166,22,113011.375000
5555,0,1.908339,0.368705,0.236607,0.146140,0.021759,210497.765625,0.071563,0.049858,0.069407,0.052708,0.084438,0.040196,2.957101,2.540267,0.373789,0.456937,22,68926.210938
5556,0,1.231826,0.254990,0.271303,0.090766,0.011847,224611.203125,0.110704,0.089852,0.102500,0.011317,0.026447,0.075495,1.870320,1.654624,0.371276,0.065832,22,56957.425781


In [26]:
%%time
computation = histogramming.load_and_fill(data_col = 'rljet_pt_comb',file_list = x[:20])

CPU times: user 6.24 ms, sys: 43 µs, total: 6.28 ms
Wall time: 6.06 ms


[Delayed('fill-bd40efa3-744f-4036-8e64-39d8fa5f5b9f'),
 Delayed('fill-50506a7d-5932-4b01-98fc-82f2db71d994'),
 Delayed('fill-e22353b6-4c31-4f2e-85d4-992126babbc0'),
 Delayed('fill-be4ef0df-468b-4cc4-8047-db7c1f0ea8b3'),
 Delayed('fill-b5ab8b40-67ed-4039-8740-8f171957abba'),
 Delayed('fill-47538bee-143e-4cb2-a591-85ca6557fd2f'),
 Delayed('fill-5e738449-a38c-4eb3-bce2-46efaa94310c'),
 Delayed('fill-b6642b12-cf98-4e59-b296-1ed397ce93a3'),
 Delayed('fill-3f9fc7eb-ef9b-4064-9cb7-efc5bd4b58bf'),
 Delayed('fill-bcb5483e-46f5-477e-93e3-f4fbe183009e'),
 Delayed('fill-b8dfb792-680a-4622-9d98-f0df6357069a'),
 Delayed('fill-95d050fc-4b3d-451d-a3e2-c654711a226b'),
 Delayed('fill-11e6ff44-0956-4033-8b56-9175e454679f'),
 Delayed('fill-7456a2c9-6316-4bd3-9c74-85d3f0c8d54c'),
 Delayed('fill-e7f5af17-ccb9-4825-b339-47affa2c838c'),
 Delayed('fill-e10da836-0b62-4342-92f0-2183f967c172'),
 Delayed('fill-a4b6b7e2-c1d6-4c1c-905f-0a59a1a26c62'),
 Delayed('fill-e03000a7-0f44-4489-86f4-edd4c3bbfc36'),
 Delayed('

In [27]:
%%time
temp = dask.compute(histogramming.load_and_fill(data_col = 'rljet_pt_comb',file_list = x[:20]))

CPU times: user 246 ms, sys: 55.8 ms, total: 302 ms
Wall time: 1.96 s


In [28]:
x = dask.visualize(histogramming.load_and_fill(data_col = 'leptons_pt',file_list = x[:20]))

In [29]:
10*temp[0][6] + temp[0][0]

Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double()) # Sum: 47336.0 (90047.0 with flow)

In [30]:
asizeof.asizeof(temp)/1024**2

0.006195068359375

In [31]:
histogramming.histograms['h1']

Hist(Regular(300, 0, 2e+06, name='test', label='test'), storage=Double())

In [32]:
psutil.virtual_memory().percent

33.1